In [1]:
from dataloader.BSD500 import BSD500
import torch
from R_network_relax import RNet
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 25  


val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_25_relaxed_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]


exp = "/home/pourya/mm_final/exps/sigma25/64_2_7_1e-3_relaxed"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:3'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()


model.eigenimage = model.eigenimage.to(device)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)

In [5]:
def safi_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k_new = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)
            rel_err_out = (torch.norm(c_k - c_k_new) / torch.norm(c_k)).item() 
            sc = "{:0.1e}".format(rel_err_out)
            c_k = c_k_new
    
    return c_k

In [6]:
i = 0
psnrs_all = list()

with torch.no_grad():
    for runs in range(5):
        psnrs = list()
        for img in test_dataset:
            i = i + 1
            if True:
                gt = img.to(device)[None, :, :, :]
                noisy_image = (img.to(device) + (25/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
                denoised =safi_denoise(noisy_image)
                psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
                psnrs.append(psnr)
                print(i, psnr)
                
        psnrs_all.append(psnrs)

1 29.91
2 28.53
3 30.61
4 29.76
5 29.98
6 26.47
7 28.94
8 28.58
9 29.8
10 28.94
11 28.16
12 26.15
13 29.51
14 26.62
15 28.16
16 30.73
17 28.28
18 29.26
19 31.53
20 29.54
21 33.02
22 27.34
23 26.67
24 24.38
25 27.87
26 28.86
27 27.86
28 32.99
29 26.07
30 30.35
31 27.92
32 32.03
33 30.04
34 23.94
35 28.52
36 28.31
37 29.08
38 27.52
39 33.55
40 26.33
41 30.89
42 32.81
43 30.89
44 30.35
45 27.06
46 29.68
47 28.88
48 26.87
49 25.86
50 38.38
51 27.51
52 33.46
53 33.64
54 30.17
55 29.02
56 27.18
57 26.26
58 27.48
59 24.45
60 26.56
61 28.56
62 28.04
63 30.93
64 24.67
65 28.76
66 32.9
67 32.87
68 29.46
69 29.96
70 28.57
71 30.55
72 29.78
73 29.96
74 26.56
75 28.9
76 28.67
77 29.81
78 28.92
79 28.16
80 26.15
81 29.48
82 26.56
83 28.12
84 30.76
85 28.26
86 29.29
87 31.43
88 29.54
89 33.13
90 27.38
91 26.69
92 24.35
93 27.83
94 28.76
95 27.84
96 33.01
97 26.04
98 30.32
99 27.86
100 32.04
101 30.0
102 23.95
103 28.58
104 28.28
105 29.09
106 27.52
107 33.52
108 26.29
109 30.79
110 32.67
111 30.89
11

In [8]:
psnr_mat = np.zeros((5, 68))

for i in range(5):
    psnr_mat[i, :] = np.array(psnrs_all[i])

std_vec = np.std(psnr_mat, axis=0)
avg_vec = np.mean(psnr_mat, axis=0)

print('std mean:', np.mean(std_vec))
print('mean mean:', np.mean(avg_vec))

std mean: 0.02667203511765261
mean mean: 29.04938235294118
